In [1]:
import torch 
import math
import torch.nn as nn    

In [2]:
class InputEmbeddings(nn.Module):
    # d_model: the dimension of the embeddings: 512
    # vocab_size: the size of the vocabulary
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)
    
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

In [3]:
class PositionalEncoding(nn.Module):
    # d_model: the dimension of the embeddings: 512
    # seq_len: the length of the sentence: 1024
    # dropout: the dropout rate: 0.1
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        for pos in range(seq_len):
            for i in range(d_model):
                if i % 2 == 0:
                    # use log to avoid overflow
                    pe[pos][i] = math.sin(pos * (-1) * math.log(10000) * (i / d_model))
                else:
                    pe[pos][i] = math.cos(pos * (-1) * math.log(10000) * ((i-1)/ d_model))
            
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.size(1), :]).requires_grad_(False)
        return self.dropout(x)

In [4]:
class LayerNormalization(nn.Module):
    def __init__(self, eps: float = 1e-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1)) # multiply by alpha
        self.bias = nn.Parameter(torch.zeros(1)) # add bias

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True) 
        std = x.std(dim=-1, keepdim=True) 
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


In [5]:
class FeedForwardBlock(nn.Module):
    # d_model: the dimension of the embeddings: 512
    # d_ff: the dimension of the feed forward network: 2048
    # dropout: the dropout rate: 0.1
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # W1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # W2 and b2

    def forward(self, x):
        # x is of shape (batch_size, seq_len, d_model)
        return self.linear_2(
            self.dropout(
                torch.relu(
                    self.linear_1(x)
                )
            )
        )

In [6]:
class MultiHeadAttentionBlock(nn.Module):
    # d_model: the dimension of the embeddings: 512
    # num_heads: the number of heads: 8
    # dropout: the dropout rate: 0.1
    def __init__(self, d_model: int, num_heads: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.w_q = nn.Linear(d_model, d_model) # Wq
        self.w_k = nn.Linear(d_model, d_model) # Wk
        self.w_v = nn.Linear(d_model, d_model) # Wv
        self.w_o = nn.Linear(d_model, d_model) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask=None, dropout: nn.Dropout=None):
        head_dim = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(head_dim)
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(attention_scores, dim=-1)
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        output = attention_scores @ value
        return output, attention_scores
        

    def forward(self, q, k, v, mask=None):
        query = self.w_q(q) # (batch_size, seq_len, d_model) ---> (batch_size, seq_len, d_model)
        key = self.w_k(k) # (batch_size, seq_len, d_model) ---> (batch_size, seq_len, d_model)
        value = self.w_v(v) # (batch_size, seq_len, d_model) ---> (batch_size, seq_len, d_model)

        # query = (batch_size, seq_len, d_model) --> (batch_size, seq_len, num_heads, head_dim) --> (batch_size, num_heads, seq_len, head_dim)
        query = query.view(query.shape[0], 
                           query.shape[1], 
                           self.num_heads, 
                           self.head_dim
                        ).transpose(1, 2) # (batch_size, num_heads, seq_len, head_dim)
        key = key.view(key.shape[0], 
                       key.shape[1], 
                       self.num_heads, 
                       self.head_dim
                        ).transpose(1, 2) # (batch_size, num_heads, seq_len, head_dim)
        value = value.view(value.shape[0], 
                           value.shape[1], 
                           self.num_heads, 
                           self.head_dim
                        ).transpose(1, 2) # (batch_size, num_heads, seq_len, head_dim)
        
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, self.dropout)

        x = x.transpose(1, 2) # (batch_size, seq_len, num_heads, head_dim) --> (batch_size, seq_len, d_model)
        x = x.contiguous().view(x.shape[0], -1, self.d_model) # (batch_size, seq_len, d_model)

        return self.w_o(x)

In [7]:

class ResidualConnection(nn.Module):
    def __init__(self, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()
        
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [8]:
class EncoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, 
                 feed_forward_block: FeedForwardBlock, 
                 dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connection = nn.ModuleList([
            ResidualConnection(dropout) for _ in range(2)
        ])

    def forward(self, x, src_mask=None):
        x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connection[1](x, self.feed_forward_block)
        return x

In [ ]:

class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, 
                 cross_attention_block: MultiHeadAttentionBlock, 
                 feed_forward_block: FeedForwardBlock, 
                 dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connection = nn.ModuleList([
            ResidualConnection(dropout) for _ in range(3)
        ])

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connection[1](x, lambda x: self.cross_attention_block(x, 
                                                                                encoder_output, 
                                                                                encoder_output, 
                                                                                src_mask))
        x = self.residual_connection[2](x, self.feed_forward_block)
        return x

In [ ]:

class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()
        
    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.proj = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        return torch.log_softmax(self.proj(x), dim=-1)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, 
                 encoder: Encoder, 
                 decoder: Decoder, 
                 src_embed: InputEmbeddings, 
                 tgt_embed: InputEmbeddings, 
                 src_pos_embed: PositionalEncoding, 
                 tgt_pos_embed: PositionalEncoding, 
                 projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos_embed = src_pos_embed
        self.tgt_pos_embed = tgt_pos_embed
        self.projection_layer = projection_layer
        
    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos_embed(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos_embed(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        return self.projection_layer(x)

In [ ]:
def build_transformer(src_vocab_size: int, 
                      tgt_vocab_size: int, 
                      src_seq_len: int, 
                      tgt_seq_len: int, 
                      d_model: int, 
                      num_layers: int = 6,
                      num_heads: int = 8,  
                      d_ff: int = 2048, 
                      dropout: float = 0.1) -> Transformer:
    
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding
    src_pos_embed = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos_embed = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(num_layers):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, num_heads, dropout)
        encoder_feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_blocks.append(EncoderBlock(encoder_self_attention_block, encoder_feed_forward_block, dropout))
    
    encoder = Encoder(nn.ModuleList(encoder_blocks))

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(num_layers):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, num_heads, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, num_heads, dropout)
        decoder_feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_blocks.append(DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, decoder_feed_forward_block, dropout))
    
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(encoder, 
                              decoder, 
                              src_embed,
                              tgt_embed, 
                              src_pos_embed, 
                              tgt_pos_embed, 
                              projection_layer
                            )
    

    # intialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer